In [274]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import euclidean_distances
train = np.load('./dlib_feature.npy')
print(len(train))
train = np.hstack((train,np.load('./dlib_feature_train.npy')))
train = list(filter(lambda s:len(s)>0,train))
print(len(train))
val = np.load('./dlib_feature_val.npy')
val= list(filter(lambda s:len(s)>0,val))
print(len(val))

30
566
56


In [275]:
clf = KNeighborsClassifier(10,'distance')

In [276]:
x_clf = []
y_clf = []
for i,ob in enumerate(train):
    for j in ob[:2]:
        x_clf.append(j)
        y_clf.append(i)

In [277]:
clf.fit(x_clf,y_clf)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='distance')

In [278]:
class my_cla(object):
    def __init__(self,arr):
        self.vectors = [np.mean(i,axis=0) for i in arr]
    def predict(self,x_in):
        return [np.array(list(map(lambda v:euclidean_distances(x_in.reshape(1,-1),v.reshape(1,-1)),self.vectors))).argmin()]
    def predict_proba(self,x_in):
        return np.array([np.array(list(map(lambda v:euclidean_distances(x_in.reshape(1,-1),v.reshape(1,-1)),self.vectors))).min()])
my_clf = my_cla(train)
my_clf.predict(x_clf[1])

[0]

In [279]:
from copy import deepcopy
def index_swap(arr,arr_id):
    print(len(arr) == len(arr_id))
    temp = deepcopy(arr)
    for i,ii in enumerate(arr_id):
        temp[i] = arr[ii]
    return temp

In [280]:
x_log = []
y_log = []
x_log_train_log = []
y_log_train_log = []
x_log_test_log = []
y_log_test_log = []
for i,ob in enumerate(train):
    for j in ob[2:]:
        x_log.append(j)
        y_log.append(i)

        from random import shuffle
arr_id = list(range(len(x_log)))
shuffle(arr_id)
x_log = index_swap(x_log,arr_id)
y_log = index_swap(y_log,arr_id)

x_log_other = []
for i in val:
    x_log_other += i
shuffle(x_log_other)
x_log_other = x_log_other

len_x_log = len(x_log)
len_x_log_other = len(x_log_other)
len_train1 = 50#int(len_x_log/5*1) + 1
len_train2 = 50#int(len_x_log_other/5*1) +1
y_log_other = [-1]*len_x_log_other
x_log_train_log += x_log[:len_train1]+x_log_other[:len_train2]
y_log_train_log += y_log[:len_train1]+y_log_other[:len_train2]
x_log_test_log += x_log[len_train1:]+x_log_other[len_train2:]
y_log_test_log += y_log[len_train1:]+y_log_other[len_train2:]
print('subject of train:{}'.format(len(set(y_log_train_log))))
print('subject of test:{}'.format(len(set(y_log_test_log))))
print(len(x_log_train_log),len(x_log_test_log))
print(len(y_log_train_log),len(y_log_test_log))

True
True
subject of train:47
subject of test:565
100 5699
100 5699


In [281]:
clf.score(x_log,y_log)

0.96058178752107926

In [282]:
clf.predict_proba(x_log[8].reshape(1,-1)).max()

0.23764773681902662

In [283]:
x_logit = []
y_logit = []
for i in x_log_train_log:
    x_logit.append([clf.predict_proba(i.reshape(1,-1)).max()])
for i in y_log_train_log:
    if i != -1:
        y_logit.append(1)
    else:
        y_logit.append(0)
x_logit = np.array(x_logit)
y_logit = np.array(y_logit)

In [284]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(C=1000000,max_iter=1000,n_jobs=4)
print(len(x_logit))
print(len(y_logit))
logit.fit(x_logit,y_logit)
a = logit.predict(x_logit)
print(logit.get_params(deep=True))
print(1/(logit.coef_[0][0]/logit.intercept_[0]))
print(logit.intercept_)
from sklearn.metrics import accuracy_score
accuracy_score(a,y_logit)

100
100
{'C': 1000000, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'max_iter': 1000, 'multi_class': 'ovr', 'n_jobs': 4, 'penalty': 'l2', 'random_state': None, 'solver': 'liblinear', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
-0.226655180696
[-20.06659975]


0.94999999999999996

In [288]:
def predict_by_joint_model(x):
    x = x.reshape(1,-1)
    res = clf.predict(x)[0]
    #res = my_clf.predict(x)[0]
    #if logit.predict(clf.predict_proba(x).max()):
    if clf.predict_proba(x).max() > 0.2127912:
    #if logit.predict(my_clf.predict_proba(x).reshape(1,-1)):
        return res
    return -1
from time import time
t = time()
print(predict_by_joint_model(x_log_test_log[0]))
val_pred = list(map(predict_by_joint_model,x_log_test_log))
print((time()-t)/len(val_pred))
print(accuracy_score(val_pred,y_log_test_log))
train_pred = list(map(predict_by_joint_model,x_log_train_log))
print(accuracy_score(train_pred,y_log_train_log))

-1
0.001450216002496926
0.921214248114
0.9


In [67]:
t =  106.45625451/-23.51944344
print(1/t)

-0.2209305930239232
